In [241]:
import glob
import gzip
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy import stats
import statsmodels.stats.multitest as multitest
import sys

from Bio.Seq import Seq
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as offline
import seaborn as sns

import matrix_transform
import visualize

%matplotlib inline
sns.set(font="Arial")
sns.set_theme(style="ticks")
colors = ['#D81B60', '#1E88E5', '#FFC107', '#31B547']
sns.set_palette(sns.color_palette(colors))

In [421]:
sets = [1, 2, 3, 4, 5, 6, 7,8, 9, 10, 11, 12, 13,\
        14, 15, 16, 17, 18, 19, 20, 21, 'R1']
res_redo = ['8R', '9R1', '9R2', '10R1', '10R2',\
            '13R1', '13R2', '14R', '16R']
all_sets = [1, 2, 3, 4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 
        17, 18, 19, 20, 21, 8, 9, 10, 'R1', '8R', '9R1', '9R2',
        '10R1', '10R2', '13R1', '13R2', '14R', '16R']
grouped_aa = ['H', 'K','R','D','E','C','M','N','Q','S','T','A',\
             'I','L','V','F','W','Y','G','P','*']
grouped_aa_r = ['*', 'P', 'G', 'Y', 'W', 'F', 'V', 'L', 'I',\
               'A', 'T', 'S', 'Q', 'N', 'M', 'C', 'E', 'D', 'R',\
               'K', 'H']

wt_ = ('SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDM'
       'LNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTP'
       'KYKFVRIQPGQTFSVLFLNGSCGSVG'
       'FNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTT'
       'ITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDI'
       'LGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ*')
wt_full = ('MSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICT'
           'SEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKV'
           'DTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIK'
           'GSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYG'
           'PFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLND'
           'FNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNG'
           'MNGRTILGSALLEDEFTPFDVVRQCSGVTFQ')
wt_ = [x for x in wt_]
wt_full = [x for x in wt_full]

### Raw counts

In [8]:
glu_dir = 'Data/glu_count_matrices/'
gal_dir = 'Data/gal_count_matrices/'
gc_dir = 'Data/gc_count_matrices/'
grl_dir = 'Data/grl_count_matrices/'

In [60]:
for res in range(1, 307):
    reps = []
    for replicate in [0, 1]:
        combined_df = []
        for path in [glu_dir, gal_dir, gc_dir, grl_dir]:
            path_dir = path + "*rep_" + str(replicate) + "residue" + str(res) + ".csv"
            res_data = pd.read_csv(glob.glob(path_dir)[0], index_col = 0)
            res_data = res_data.set_index(['site_1', 'site_2', 'site_3'])
            combined_df.append(res_data)
        concat_df = pd.concat(combined_df, axis = 1, sort = False)
        concat_df.columns = ['glu', 'gal', 'gc', 'grl']
        reps.append(concat_df)
    reps_df = pd.concat(reps, axis = 1, sort = False)
    reps_df.columns = ['glu1', 'gal1', 'gc1', 'grl1', 'glu2', 'gal2', 'gc2', 'grl2']
    reps_df.to_csv('Data/combined_raw_counts/res'+str(res)+'.csv')

### Foldchanges

In [65]:
glu_gal = 'Data/count_matrix_glu_gal/'
glu_gc = 'Data/count_matrix_glu_gc/'
glu_grl = 'Data/count_matrix_glu_grl/'

In [81]:
for res in range(1, 307):
    reps = []
    for replicate in [0, 1]:
        combined_df = []
        for path in [glu_gal, glu_gc, glu_grl]:
            path_dir = path + "set" + str(res) + "_rep" + str(replicate) + "*.csv"
            res_data = pd.read_csv(glob.glob(path_dir)[0], index_col = 0)
            res_data = res_data.set_index(['site_1', 'site_2', 'site_3'])
            combined_df.append(res_data)
        concat_df = pd.concat(combined_df, axis = 1, sort = False)
        concat_df.columns = ['glu', 'gal', 'glu_gal', 'glu', 'gc', 
                             'glu_gc', 'glu', 'grl', 'glu_grl']
        reps.append(concat_df)
    reps_df = pd.concat(reps, axis = 1, sort = False)
    reps_df.columns = ['glu1', 'gal1', 'glu_gal1', 'glu1', 'gc1', 
                       'glu_gc1', 'glu1', 'grl1', 'glu_grl1',
                       'glu2', 'gal2', 'glu_gal2', 'glu2', 'gc2', 
                       'glu_gc2', 'glu2', 'grl2', 'glu_grl2']
    reps_df.to_csv('Data/combined_raw_foldchange/res'+str(res)+'.csv')

### Combine raw counts and foldchanges

In [113]:
for res in range(1, 307):
    counts = pd.read_csv('Data/combined_raw_counts/res'+str(res)+'.csv')
    foldchange = pd.read_csv('Data/combined_raw_foldchange/res'+str(res)+'.csv')
    merged = counts.merge(foldchange, on = ['site_1', 'site_2', 'site_3'], how = 'outer',
             suffixes=('_counts', '_foldchange'))
    merged['combined'] = merged['site_1']+merged['site_2']+merged['site_3']
    merged = merged[~merged["combined"].str.contains('\n')]
    merged = merged[~merged["combined"].str.contains('N')]
    merged['AA'] = merged['combined'].apply(lambda x: Seq(x).translate())
    merged.to_csv('Data/combined_raw_count_foldchange/res'+str(res)+'.csv')

### Raw foldchanges by amino acid

In [311]:
def f(x, y):
    stat, pval = stats.ttest_ind(np.array(x), 
                           np.array(y), equal_var = False)
    return stat, pval

glu_res = []
for res in range(1, 307):
    glu_gal = pd.read_csv(glob.glob('Data/matrix_vals_glu_gal/res'+str(res)+'_glu_gal.csv')[0])
    glu_gc = pd.read_csv(glob.glob('Data/matrix_vals_glu_gc/res'+str(res)+'_glu_gc.csv')[0])
    glu_grl = pd.read_csv(glob.glob('Data/matrix_vals_glu_grl/res'+str(res)+'_glu_grl.csv')[0])
    glu_gal['list'] = glu_gal['mean'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_gc['list'] = glu_gc['mean'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_grl['list'] = glu_grl['mean'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_gal = glu_gal.set_index('Translation')
    glu_grl = glu_grl.set_index('Translation')
    glu_gc = glu_gc.set_index('Translation')
    concat_glu = pd.concat([glu_gal, glu_grl, glu_gc], axis = 1)
    concat_glu.columns = ['mean_glu_gal', 'list_glu_gal', 
                         'mean_glu_grl', 'list_glu_grl',
                         'mean_glu_gc', 'list_glu_gc']
    concat_glu['grl_pval'] = concat_glu.apply(lambda x: f(x['list_glu_gal'], x['list_glu_grl'])[1], axis=1)
    concat_glu['gc_pval'] = concat_glu.apply(lambda x: f(x['list_glu_gal'], x['list_glu_gc'])[1], axis=1)
    concat_glu['grl_stat'] = concat_glu.apply(lambda x: f(x['list_glu_gal'], x['list_glu_grl'])[0], axis=1)
    concat_glu['gc_stat'] = concat_glu.apply(lambda x: f(x['list_glu_gal'], x['list_glu_gc'])[0], axis=1)
    concat_glu['residue'] = len(concat_glu)*[res]
    glu_res.append(concat_glu)
    
glu_res_concat = pd.concat(glu_res)
glu_res_concat['grl_pval_correct'] = multitest.multipletests(glu_res_concat['grl_pval'])[1]
glu_res_concat['gc_pval_correct'] = multitest.multipletests(glu_res_concat['gc_pval'])[1]

/Users/jennysheng/anaconda3/envs/tools/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3506: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/jennysheng/anaconda3/envs/tools/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/jennysheng/anaconda3/envs/tools/lib/python3.6/site-packages/statsmodels/stats/multitest.py:160: RuntimeWarning:

invalid value encountered in greater

/Users/jennysheng/anaconda3/envs/tools/lib/python3.6/site-packages/statsmodels/stats/multitest.py:254: RuntimeWarning:

invalid value encountered in greater



In [340]:
grl_vars = glu_res_concat[glu_res_concat['grl_stat']>0]
fig = px.scatter(grl_vars, x = 'residue', y = -np.log10(grl_vars['grl_pval']))
fig.show()
offline.plot(fig, filename = 'Figures/grl_manhattan.html')

'Figures/grl_manhattan.html'

In [341]:
gc_vars = glu_res_concat[glu_res_concat['gc_stat']>0]
fig = px.scatter(gc_vars, x = 'residue', y = -np.log10(gc_vars['gc_pval']))
fig.show()
offline.plot(fig, filename = 'Figures/gc_manhattan.html')

'Figures/gc_manhattan.html'

### Separate biological replicates

In [381]:
def f(x, y):
    stat, pval = stats.ttest_ind(np.array(x), 
                           np.array(y), equal_var = False)
    return stat, pval

glu_res = []
for res in range(1, 307):
    glu_gal = pd.read_csv(glob.glob('Data/matrix_vals_glu_gal_separate/res'+str(res)+'_glu_gal.csv')[0])
    glu_gc = pd.read_csv(glob.glob('Data/matrix_vals_glu_gc_separate/res'+str(res)+'_glu_gc.csv')[0])
    glu_gal['list_x'] = glu_gal['ratio_x'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_gal['list_y'] = glu_gal['ratio_y'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_gc['list_x'] = glu_gc['ratio_x'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_gc['list_y'] = glu_gc['ratio_y'].apply(lambda x: 
                x[1:-1].split(',')).apply(lambda x: [float(i) for i in x])
    glu_gal = glu_gal.set_index('Translation')
    glu_gc = glu_gc.set_index('Translation')
    concat_glu = pd.concat([glu_gal, glu_gc], axis = 1)
    concat_glu.columns = ['mean_glu_gal_x', 'mean_glu_gal_y','list_glu_gal_x', 'list_glu_gal_y',
                         'mean_glu_gc_x', 'mean_glu_gc_x','list_glu_gc_x', 'list_glu_gc_y']
    concat_glu_gc = pd.DataFrame()
    concat_glu_gc['glu_gal'] = concat_glu['list_glu_gal_x'] + concat_glu['list_glu_gal_y']
    concat_glu_gc['glu_gc'] = concat_glu['list_glu_gc_x'] + concat_glu['list_glu_gc_y']
    concat_glu_gc['gc_pval'] = concat_glu_gc.apply(lambda x: f(x['glu_gal'], x['glu_gc'])[1], axis=1)
    concat_glu_gc['gc_stat'] = concat_glu_gc.apply(lambda x: f(x['glu_gal'], x['glu_gc'])[0], axis=1)
    concat_glu_gc['residue'] = len(concat_glu_gc)*[res]
    glu_res.append(concat_glu_gc)
    
glu_res_concat = pd.concat(glu_res)
# glu_res_concat['grl_pval_correct'] = multitest.multipletests(glu_res_concat['grl_pval'])[1]
glu_res_concat['gc_pval_correct'] = multitest.multipletests(glu_res_concat['gc_pval'])[1]

In [384]:
glu_res_concat = pd.concat(glu_res)
glu_res_concat['gc_pval_correct'] = multitest.multipletests(glu_res_concat['gc_pval'])[1]

/Users/jennysheng/anaconda3/envs/tools/lib/python3.6/site-packages/statsmodels/stats/multitest.py:160: RuntimeWarning:

invalid value encountered in greater

/Users/jennysheng/anaconda3/envs/tools/lib/python3.6/site-packages/statsmodels/stats/multitest.py:254: RuntimeWarning:

invalid value encountered in greater



In [397]:
gc_vars = glu_res_concat[glu_res_concat['gc_stat']>0]
fig = px.scatter(gc_vars, x = 'residue', y = -np.log10(gc_vars['gc_pval']))
fig.show()
offline.plot(fig, filename = 'Figures/gc_manhattan.html')

'Figures/gc_manhattan.html'

### gc/gal and grl/gal

In [417]:
sample_dir = 'sample_spreadsheet_021521.csv'
samples = pd.read_csv(sample_dir, comment = '#')
fig_folder = 'Figures/'

In [450]:
all_residues_grl, mean_stop_dict = matrix_transform.transform_matrix_sigma(
            'grl_gal', 'grl_gal.csv', samples, sets, res_redo, all_sets)
visualize.make_heatmap(all_residues_grl, [int(x[4:]) for x in list(all_residues_grl.columns)], \
            grouped_aa, wt_full[1:], show = True, 
            save = False, name = fig_folder+'grl_gal_standardize')
all_residues_grl.to_csv('CSVs/grl_gal.csv')

In [451]:
all_residues_gc, mean_stop_dict = matrix_transform.transform_matrix_sigma(
            'gc_gal', 'gc_gal.csv', samples, sets, res_redo, all_sets)
all_residues_gc.to_csv('CSVs/gc_gal.csv')
all_residues_gc['Res 121'] = [0]*21
visualize.make_heatmap(all_residues_gc, [int(x[4:]) for x in list(all_residues_gc.columns)], \
            grouped_aa, wt_full[1:], show = True, 
            save = False, name = fig_folder+'gc_gal_standardize')

In [459]:
all_residues, mean_stop_dict = matrix_transform.transform_matrix_sigma(
            'new_glu_gal', 'gal_glu.csv', samples, sets, res_redo, all_sets)
visualize.make_heatmap(all_residues, [int(x[4:]) for x in list(all_residues.columns)], \
            grouped_aa, wt_full[1:], show = True, 
            save = False, name = fig_folder+'glu_gal_standardize')

In [460]:
all_residues

,Res 1,Res 2,Res 3,Res 4,Res 5,Res 6,Res 7,Res 8,Res 9,Res 10,...,Res 297,Res 298,Res 299,Res 300,Res 301,Res 302,Res 303,Res 304,Res 305,Res 306
*,-1.817541,-2.235454,-2.407931,-2.012656,-2.430530,-1.981676,-2.039062,-2.150798,-2.129819,-2.256642,...,-1.207806,-1.898364,-3.827833,-3.908609,-0.185186,-0.976392,-0.261688,0.382787,-0.970729,0.376992
A,0.250942,-0.489973,-1.939774,-2.019003,0.329928,-1.591490,0.003031,0.277117,-2.084479,-2.222721,...,-0.185623,-0.932023,-2.494991,-0.191059,0.047359,0.004237,-0.168236,0.103338,-0.281665,0.029806
C,0.070121,-0.586546,-2.348899,-0.704170,0.188813,-1.391194,0.133115,0.510012,-1.894139,-1.733627,...,0.705963,-0.833554,-1.493578,-0.173312,0.213964,0.092682,-0.099832,-0.009731,-0.076118,0.178903
D,-2.435644,-1.973016,-1.909095,-2.120174,-2.227110,-2.092321,-1.935589,-2.022860,-2.326049,-1.905508,...,0.416171,-1.923958,-3.839802,-2.938257,-0.022935,-0.412897,0.079475,-0.083471,-0.680422,-0.016971
E,-2.552269,-1.929714,-2.192538,-2.231286,-2.260891,-1.742233,-2.251470,-1.495570,-1.760655,-1.715465,...,-0.667082,-2.655395,-3.720217,-1.697194,0.314997,-0.089676,0.094348,0.154533,-0.317691,-0.073158
F,-0.882827,-0.928799,-0.173888,-2.193377,0.806827,-0.231107,-2.019990,0.281887,-2.376388,-2.246397,...,0.523383,0.149636,-1.382179,0.606036,0.280118,0.473622,0.395289,-0.093955,-0.194934,-0.017288
G,0.207975,0.222772,-1.971897,-2.407347,-1.762307,-2.238330,-2.119819,-2.247801,-2.310252,-2.125334,...,-0.007177,-2.037103,-2.785162,-0.490680,-0.244033,0.163413,-0.009736,0.016002,-0.231332,0.000773
H,0.933556,-0.457832,-2.409195,-0.659616,0.211607,-2.451255,-2.233141,0.344344,-1.837074,-2.153134,...,0.069085,0.381648,-0.099622,-0.165134,0.055528,0.545395,-0.199862,0.104719,-0.113866,0.153254
I,-0.114652,-0.679276,-0.175678,0.745904,0.230954,-0.471646,-1.870006,-1.980712,-2.114630,-2.193613,...,0.205311,-0.875645,0.117061,-0.001365,0.075880,-0.457002,0.450418,0.284194,0.209859,-0.568129
K,0.544170,-0.411713,-2.317152,0.155622,0.104208,-2.117626,-2.090284,-2.043832,-1.947212,-2.093592,...,0.325244,0.292207,-3.736995,-0.646972,0.230016,0.863805,0.039761,0.442595,-0.385550,-0.415365
